In [1]:
# Langsmith with vector test
''' https://www.langchain.com/langsmith '''
''' reference : https://hellollama.net/llm-%EC%B4%88%EA%B8%89%EA%B0%95%EC%A2%8C-7-langsmith-%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0/'''

' reference : https://hellollama.net/llm-%EC%B4%88%EA%B8%89%EA%B0%95%EC%A2%8C-7-langsmith-%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0/'

In [2]:
''' langsmith .env'''
'''
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="<your-api-key>"
LANGCHAIN_PROJECT=""
'''

'\nLANGCHAIN_TRACING_V2=true\nLANGCHAIN_ENDPOINT="https://api.smith.langchain.com"\nLANGCHAIN_API_KEY="<your-api-key>"\nLANGCHAIN_PROJECT=""\n'

In [3]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

/Users/euiyoung.hwang/ES/Python_Workspace/Vector_DB_with_LLM/.venv/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
import sys
# sys.path.append("C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Langchain/workflow/")
sys.path.append("/Users/euiyoung.hwang/ES/Python_Workspace/Vector_DB_with_LLM/Langchain/workflow")
''' my langchain document loader package'''
from text_loader import loader_text

ModuleNotFoundError: No module named 'text_loader'

In [ ]:
# path = "C://Users/euiyoung.hwang/Git_Workspace/Vector_DB_with_LLM/Data/"
path = "/Users/euiyoung.hwang/ES/Python_Workspace/Vector_DB_with_LLM/Data/"

In [ ]:
splits = loader_text("{}/{}".format(path, "Sample.hwp"), create_json=False)
print(f"Langchain Document : {splits}")